# Toronto Web Scraping Code

## Get the Source from WikiPedia page

## Imports

In [33]:
# standard computing libraries
import pandas as pd
import numpy as np
import json

# webscraping
from bs4 import BeautifulSoup
import requests


# k-means clustering
from sklearn.cluster import KMeans



# library to access overpass api in a more convenient way
import overpy

# ipython command to use matplotlib
%matplotlib inline

ModuleNotFoundError: No module named 'overpy'

In [ ]:
!pip install geocoder


In [24]:
# standard computing libraries
import pandas as pd
import numpy as np
import json

# webscraping
from bs4 import BeautifulSoup
import requests

# k-means clustering
from sklearn.cluster import KMeans

# folium for maps
import folium

# geopandas for advanced geojson handling
import geopandas

# library to access overpass api in a more convenient way
import overpy

# ipython command to use matplotlib
%matplotlib inline

ModuleNotFoundError: No module named 'overpy'

In [34]:
! pip install folium

In [35]:
# ipython command to use matplotlib
%matplotlib inline

In [36]:
! pip install geopandas

In [37]:
postcode = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [38]:

postcodesoup = BeautifulSoup(postcode.content)

In [39]:
list_postcodes = list()

for tr in postcodesoup.find_all("tr")[2:-5]:
    
    row = (td.text for td in tr.find_all("td"))
    
    list_postcodes.append(row)

## use pandas to read the table into a pandas dataframe.

In [40]:

df_raw = pd.DataFrame(list_postcodes, columns=["Postcode","Borough","Neighbourhood"])

In [41]:
def concatNeighbourhood(row):
    
    return pd.Series(dict(Borough = row['Borough'].unique()[0],
                        Neighbourhood = ', '.join(row['Neighbourhood'])))

In [42]:
df_post = (df_raw
            .query('Borough != "Not assigned"')
            .assign(Neighbourhood = lambda x: x["Neighbourhood"].str[:-1].replace('Not assigned', x["Borough"]))
            .groupby('Postcode').apply(concatNeighbourhood)
            .reset_index()
)

df_post.head()

,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [43]:
df_post.shape

(178, 3)

In [44]:
df_post.head(30)

,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae
5,M1J\n,Scarborough\n,Scarborough Village
6,M1K\n,Scarborough\n,Kennedy Park / Ionview / East Birchmount Park
7,M1L\n,Scarborough\n,Golden Mile / Clairlea / Oakridge
8,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village West
9,M1N\n,Scarborough\n,Birch Cliff / Cliffside West


In [46]:
df_post.shape

(178, 3)

##  Getting the Latitude and Longitude 

In [47]:

#postal_code = 'M5G'

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [53]:
import panda as pd


ModuleNotFoundError: No module named 'panda'

In [54]:
import pandas as pd


In [56]:
import pandas as pd

In [57]:
csv_path = 'http://cocl.us/Geospatial_data'

In [58]:
data = pd.read_csv(csv_path)

In [59]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:

df_post_coord = (pd.merge(df_post, data , left_on="Postcode", right_on="Postal Code")
                   .drop("Postal Code", axis=1)
                   .rename(columns={"Postcode":"PostalCode"}))

df_post_coord.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


In [61]:
df_post_coord.head(20)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


In [63]:
df_post = (df_raw
            .query('Borough != "Not assigned"')
            .assign(Neighbourhood = lambda x: x["Neighbourhood"].str[:-1].replace('Not assigned', x["Borough"]))
            .groupby('Postcode').apply(concatNeighbourhood)
            .reset_index()
)

df_post.head()

,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [64]:

df_post_coord = (pd.merge(df_post, data , left_on="Postcode", right_on="Postal Code")
                   .drop("Postal Code", axis=1)
                   .rename(columns={"Postcode":"PostalCode"}))

df_post_coord.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


In [65]:
df_post_coord.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


In [66]:
df_post.head()

,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [67]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [68]:
df_post_coord = (pd.merge(df_post, df_coord, left_on="Postcode", right_on="Postal Code")
                   .drop("Postal Code", axis=1)
                   .rename(columns={"Postcode":"PostalCode"}))

df_post_coord.head()

NameError: name 'df_coord' is not defined

In [69]:
df_post_coord = (pd.merge(df_post, data, left_on="Postcode", right_on="Postal Code")
                   .drop("Postal Code", axis=1)
                   .rename(columns={"Postcode":"PostalCode"}))

df_post_coord.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


In [70]:

df_toronto = df_post_coord[df_post_coord['Borough'].str.contains('Toronto')]

df_toronto.shape

(0, 5)

In [71]:
df_post_coord = pd.merge(df_post, data , left_on="Postcode", right_on="Postal Code")
                
df_post_coord.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude


In [72]:

df_post.Postcode = df_post.Postcode.astype(str)
data.Postal Code = data.Postal Code.astype(str)

SyntaxError: invalid syntax (<ipython-input-72-f5bbefbd4dab>, line 3)

In [73]:
df_post.Postcode = df_post.Postcode.astype(str)

In [74]:
df_post_coord = pd.merge(df_post, data , left_on="Postcode", right_on="Postal Code")
                
df_post_coord.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude


In [76]:
data['Postal Code'] = data['Postal Code'].astype(str)

In [77]:
 df_post_coord = pd.merge(df_post, data , left_on="Postcode", right_on="Postal Code")

In [78]:
 df_post_coord.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude


In [79]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
df_post.head()


,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [81]:
df_post = df_post.replace('\n','', regex=True)

In [82]:
df_post.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## ---------------------------------------------------------------------------------------------------------------------------------------

## Merging the Two DataFrames to display Latitude and Longitude

In [83]:
df_post_coord = (pd.merge(df_post, data, left_on="Postcode", right_on="Postal Code")
                   .drop("Postal Code", axis=1)
                   .rename(columns={"Postcode":"PostalCode"}))

df_post_coord.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## ----------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. 

In [84]:
df_toronto = df_post_coord[df_post_coord['Borough'].str.contains('Toronto')]

df_toronto.shape

(39, 5)

In [85]:
! pip install folium

In [86]:

nh_map = folium.Map(location=(43.6815,-79.5284), zoom_start=11)

for i, nh in df_toronto.iterrows():
    folium.Circle(
        radius=500,
        location=[nh['Latitude'], nh['Longitude']],
        popup=nh['Neighbourhood'],
        color='crimson',
        fill=False,
    ).add_to(nh_map)
    
nh_map

### Assignment from Week3 Complete !!!